In [1]:
using NBInclude
@nbinclude("diffSHT_healpix.ipynb")
@nbinclude("utilities.ipynb")

# 1 - HEALPIX & UTILS FUNCTIONS TESTS

## 1.1 - Generate random maps to test

In [2]:
m = HealpixMap{Float64, RingOrder}(1024)
m.pixels[:] = 10*randn(Float64, nside2npix(1024));

In [3]:
m1 = HealpixMap{Float64, RingOrder}(1024)
m1.pixels[:] = 10*randn(Float64, nside2npix(1024));

In [4]:
eps = zeros(length(m))
eps[10]+=0.01
m_up = HealpixMap{Float64, RingOrder}(m + eps)
m_down = HealpixMap{Float64, RingOrder}(m - eps)
m_up1 = HealpixMap{Float64, RingOrder}(m1 + eps)
m_down1 = HealpixMap{Float64, RingOrder}(m1 - eps);

In [5]:
a = map2alm(m, lmax=2047)
a1 = map2alm(m1, lmax=2047);

In [6]:
eps = zeros(length(a.alm))
eps[11]+=0.01
eps1 = zeros(length(a.alm)) + zeros(length(a.alm))*im
eps1[11]+= 0.01*im
a_up = Alm{ComplexF64, Array{ComplexF64, 1}}(a.lmax, a.mmax, a.alm + eps)
a_down = Alm{ComplexF64, Array{ComplexF64, 1}}(a.lmax, a.mmax, a.alm - eps)
a_up1 = Alm{ComplexF64, Array{ComplexF64, 1}}(a1.lmax, a1.mmax, a1.alm + eps1)
a_down1 = Alm{ComplexF64, Array{ComplexF64, 1}}(a1.lmax, a1.mmax, a1.alm - eps1);

## 1.2 - sum(map::HealpixMap{T,O,AA})

In [199]:
function test1(map::HealpixMap{T,O,AA}) where{T, O <:Order, AA<:AbstractArray{T,1}}
    s = sum(map)
    return s
end

test1 (generic function with 1 method)

In [200]:
f, df = withgradient(x->test1(x), m)

(val = 96.7084804668944, grad = ([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],))

In [201]:
(test1(m_up)-test1(m_down))/0.02

0.9999999999990905

In [202]:
t_func = @benchmark test1(m)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  14.600 μs …  47.000 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     14.700 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.715 μs ± 652.915 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █    █     ▂     ▁     ▂     ▃     ▂                 ▁     ▁ ▂
  █▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁▇▁▁▁▁▁█▁▁▁▁▁█ █
  14.6 μs       Histogram: log(frequency) by time      15.6 μs <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [203]:
t_funcgrad = @benchmark withgradient(x->test1(x), m)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  22.500 μs …  4.801 ms  ┊ GC (min … max):  0.00% … 97.97%
 Time  (median):     25.500 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   36.072 μs ± 92.930 μs  ┊ GC (mean ± σ):  10.75% ±  4.37%

  ▂▆█▆▃▂                         ▃▄▅▃▁                        ▂
  ███████▇▇▇▅▇▆▅▅▅▅▃▆▆▅▄▄▅▄▃▄▃▁▄▇██████▆▇▆▆▆▆▇▄▅▄▅▄▄▄▅▅▄▄▄▅▅▅ █
  22.5 μs      Histogram: log(frequency) by time        87 μs <

 Memory estimate: 96.55 KiB, allocs estimate: 21.

In [204]:
median(t_funcgrad).time/median(t_func).time

1.7346938775510203

## 1.3 - sum(alm::Alm{ComplexF64, Array{ComplexF64, 1}})

In [205]:
function test2(alm::Alm{ComplexF64, Array{ComplexF64, 1}})
    s = real(sum(alm))
    return s
end

test2 (generic function with 1 method)

In [206]:
f, df = withgradient(x->test2(x), a)

(val = 9.658801269384805, grad = (Alm{ComplexF64, Vector{ComplexF64}}(ComplexF64[1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im  …  1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im], 95, 95, 191),))

In [207]:
(test2(a_up)-test2(a_down))/0.02

0.9999999999999787

In [208]:
t_func = @benchmark test2(a)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.470 μs …  4.780 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.490 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.498 μs ± 90.674 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▇  █ ▃       ▂  ▁         ▂                              ▁
  ▇▁█▁▁█▁█▁▁█▁▁▇▁█▁▁█▁▁▆▁▄▁▁▇▁█▁▁█▁▁▇▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁█▁▆ █
  1.47 μs      Histogram: log(frequency) by time     1.69 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [209]:
t_funcgrad = @benchmark withgradient(x->test2(x), a)

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   8.500 μs …  1.580 ms  ┊ GC (min … max):  0.00% … 97.60%
 Time  (median):     10.900 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   16.314 μs ± 41.167 μs  ┊ GC (mean ± σ):  16.12% ±  6.54%

  ▃▅██▄▂ ▁ ▁                       ▃▄▄▁                       ▂
  ██████████▇▆▆▁▆▆▄▄▁▃▁▁▁▁▄▄▅▆▁▁▁▃▆████▇▇▆▆▆▅▄▄▅▅▄▅▁▄▄▄▁▃▁▃▅▄ █
  8.5 μs       Histogram: log(frequency) by time      54.8 μs <

 Memory estimate: 73.25 KiB, allocs estimate: 21.

In [210]:
median(t_funcgrad).time/median(t_func).time

7.315436241610739

## 1.4 - conj(alm::Alm{ComplexF64, Array{ComplexF64, 1}})

In [7]:
function test3(alm::Alm{ComplexF64, Array{ComplexF64, 1}})
    s = imag(sum(conj(alm)))
    return s
end

test3 (generic function with 1 method)

In [8]:
f, df = withgradient(x->test3(x), a)

(val = -6.895758189445881, grad = (Alm{ComplexF64, Vector{ComplexF64}}(ComplexF64[0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im  …  0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im, 0.0 - 1.0im], 95, 95, 191),))

In [9]:
(test3(a_up1)-test3(a_down1))/0.02

-1.0000000000000675

In [253]:
t_func = @benchmark test3(a)

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   7.000 μs …  1.586 ms  ┊ GC (min … max):  0.00% … 98.13%
 Time  (median):      9.200 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   11.706 μs ± 31.997 μs  ┊ GC (mean ± σ):  17.18% ±  6.61%

  ▂▃▄▆██▃▁                                                 ▁  ▂
  █████████▇▇▆▆▆▆▆▆▅▄▄▄▁▄▃▁▃▁▁▁▁▃▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▃▆███ █
  7 μs         Histogram: log(frequency) by time      34.4 μs <

 Memory estimate: 72.80 KiB, allocs estimate: 2.

In [254]:
t_funcgrad = @benchmark withgradient(x->test3(x), a)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  19.300 μs …   5.039 ms  ┊ GC (min … max):  0.00% … 97.52%
 Time  (median):     27.200 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   43.422 μs ± 129.761 μs  ┊ GC (mean ± σ):  18.93% ±  6.51%

  ▁▄▆██▄▂▁                                ▁▃▄▃▁                ▂
  █████████▇▇▆▆▆▅▅▄▄▆▇▅▅▁▁▁▁▃▁▄▅▄▄▅▄▃▃▁▁▃▃███████▇▆▅▅▃▄▄▁▄▄▄▄▄ █
  19.3 μs       Histogram: log(frequency) by time       138 μs <

 Memory estimate: 218.94 KiB, allocs estimate: 27.

In [255]:
median(t_funcgrad).time/median(t_func).time

2.9565217391304346

## 1.5 - alm_prod(alm1::Alm{ComplexF64, Vector{ComplexF64}}, alm2::Alm{ComplexF64, Vector{ComplexF64}})

In [10]:
function test4(alm1::Alm{ComplexF64, Vector{ComplexF64}}, alm2::Alm{ComplexF64, Vector{ComplexF64}})
    a = alm_prod(alm1,alm2)
    s = real(sum(a))
    return s
end

test4 (generic function with 1 method)

In [11]:
f, df = withgradient(x->test4(x, a1), a)

(val = -0.36274843625944353, grad = (Alm{ComplexF64, Vector{ComplexF64}}(ComplexF64[0.253907753552854 + 0.0im, 0.4505105317672352 + 0.0im, -0.3813325647510905 + 0.0im, -0.1491264640593099 + 0.0im, -0.14318400041943039 + 0.0im, -0.2999144571577564 + 0.0im, 0.6364092510053875 + 0.0im, 0.300210131108795 + 0.0im, -0.0699701713565044 + 0.0im, 0.3163538244790232 + 0.0im  …  0.3380272422240932 - 0.02171947419062596im, -0.20102493409180447 + 0.22977877132046284im, -0.22282584031906508 - 0.12927902478126158im, -0.04074854467245283 - 0.2612505551094993im, -0.04982575377404085 - 0.2745913878799517im, 0.018941735401947993 + 0.10889935854805163im, 0.08380324675149742 - 0.17990608282901485im, 0.41232330632275505 + 0.059519797889044254im, 0.0009432200975285654 - 0.037236541700935835im, 0.10528400837979998 - 0.03698526761038631im], 95, 95, 191),))

In [12]:
df[1].alm[11]

-0.3988611311917105 + 0.0im

In [13]:
(test4(a_up,a1)-test4(a_down,a1))/0.02

-0.39886113119171895

In [256]:
t_func = @benchmark test4(a,a1)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  16.400 μs …  1.793 ms  ┊ GC (min … max):  0.00% … 95.95%
 Time  (median):     21.000 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   29.022 μs ± 91.706 μs  ┊ GC (mean ± σ):  17.05% ±  5.36%

  ▂▄▅▇█▆▂▁                                           ▂▂▁      ▂
  ████████▇▇▇▆▇▅▄▄▄▅▃▃▄▄▁▄▁▃▃▄▁▃▁▄▁▁▃▄▄▄▃▃▁▃▁▁▁▁▁▁▁▅█████▇▇▆▅ █
  16.4 μs      Histogram: log(frequency) by time      78.4 μs <

 Memory estimate: 145.69 KiB, allocs estimate: 6.

In [257]:
t_funcgrad = @benchmark withgradient(x->test4(x, a1), a)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  53.500 μs …   4.848 ms  ┊ GC (min … max):  0.00% … 95.49%
 Time  (median):     65.700 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   90.128 μs ± 188.491 μs  ┊ GC (mean ± σ):  18.82% ±  8.82%

  ▄▇█▃▁  ▁                     ▁▂▁                             ▁
  ████████▇▅▅▄▄▅▅▄▃▅▆▅▄▃▅▅▁▁▃▁▅███▇▆▅▃▅▄▄▃▅▃▃▁▄▃▃▁▁▃▄▃▄▄▄▃▃▄▁▃ █
  53.5 μs       Histogram: log(frequency) by time       377 μs <

 Memory estimate: 438.69 KiB, allocs estimate: 47.

In [258]:
median(t_funcgrad).time/median(t_func).time

3.1285714285714286

In [227]:
function test4(alm1::Alm{ComplexF64, Vector{ComplexF64}}, alm2::Alm{ComplexF64, Vector{ComplexF64}})
    a = alm_prod(alm1,alm2)
    s = imag(sum(a))
    return s
end

test4 (generic function with 1 method)

In [228]:
f, df = withgradient(x->test4(a, x), a1)

(val = -0.5308668593862856, grad = (Alm{ComplexF64, Vector{ComplexF64}}(ComplexF64[0.0 + 0.0im, 0.0 + 0.0im, -0.0 + 0.0im, 0.0 + 0.0im, -0.0 + 0.0im, -0.0 + 0.0im, 0.0 + 0.0im, -0.0 + 0.0im, -0.0 + 0.0im, -0.0 + 0.0im  …  0.5425603263892964 + 0.27241303524784194im, 0.1113988265577519 + 0.09344786273200449im, 0.06330875141086517 + 0.19681254928603478im, -0.021986949382922828 + 0.1314894754536127im, -0.09397268380503732 + 0.04247727022571366im, 0.39081218601224915 + 0.11506692290177446im, 0.1255210924225785 + 0.05817487292315588im, -0.18060994738235994 - 0.1944199088724708im, -0.2795205990535535 - 0.25288479537133746im, -0.3304277135509716 - 0.013340527413356605im], 95, 95, 191),))

In [229]:
df[1].alm[10]

-0.0 + 0.0im

In [230]:
(test4(a,a_up1)-test4(a,a_down1))/0.02

0.0

## 1.6 - alm2map(alm::Alm{ComplexF64, Vector{ComplexF64}}, nside::Integer)

In [14]:
function test5(alm::Alm{ComplexF64, Vector{ComplexF64}}, nside::Integer)
    m = alm2map(alm, nside)
    s = sum(m)
    return s
end

test5 (generic function with 1 method)

In [15]:
f, df = withgradient(x->test5(x, 32), a)

(val = 291.9329604788829, grad = (Alm{ComplexF64, Vector{ComplexF64}}(ComplexF64[3466.3808013174157 + 0.0im, 3.829412739647291e-15 + 0.0im, -0.35050351685367787 + 0.0im, -8.77428687333097e-15 + 0.0im, -0.8059313632047633 + 0.0im, 1.3748938363971676e-14 + 0.0im, -0.690103660549994 + 0.0im, -1.8731184499464162e-14 + 0.0im, -0.5300925273127177 + 0.0im, 2.3716409200848677e-14 + 0.0im  …  -2.758022265874035e-21 - 1.392938795939659e-29im, 0.0 + 0.0im, -2.499462592840128e-19 - 8.663609192724006e-28im, 0.0 + 0.0im, 3.633691577486515e-32 - 4.565484393116853e-32im, 0.0 + 0.0im, 2.6884149992965502e-30 - 3.378459903899197e-30im, 7.63040463546246e-29 - 7.409374029955763e-30im, 0.0 + 0.0im, -6.413674679094654e-29 + 1.3025071565759694e-29im], 95, 95, 191),))

In [16]:
df[1].alm[11]

-0.9082590450875614 + 0.0im

In [17]:
(test5(a_up, 32)-test5(a_down, 32))/0.02

-0.9082590450020689

In [51]:
eps = zeros(length(a.alm))
eps[113]+=0.01
a_up = Alm{ComplexF64, Array{ComplexF64, 1}}(a.lmax, a.mmax, a.alm + eps)
a_down = Alm{ComplexF64, Array{ComplexF64, 1}}(a.lmax, a.mmax, a.alm - eps)

Alm{ComplexF64, Vector{ComplexF64}}(ComplexF64[0.38668945779979985 + 0.0im, -0.27781396624029336 + 0.0im, -0.29053432593162176 + 0.0im, -0.02278182645080117 + 0.0im, -0.5813474252070974 + 0.0im, 0.6493415249691663 + 0.0im, -0.2675548315408978 + 0.0im, -0.23455593193402108 + 0.0im, 0.11661585514242741 + 0.0im, -0.5251072849365288 + 0.0im  …  -0.15188189583723125 + 0.19869801517391092im, -0.04821749802786917 + 0.13667890952207884im, 0.09717862325360682 - 0.31610595271911474im, 0.010196603471206322 + 0.6401330574869057im, -0.02613026839800301 + 0.0972650517992572im, -0.005015196888905475 + 0.06049196413651737im, -0.33397608184891814 - 0.15325919012132497im, 0.15451453427508904 - 0.0878122419309535im, 0.17603110397745164 + 0.3093460810010402im, -0.03725437911217752 - 0.09532828066424262im], 95, 95, 191)

In [52]:
(test5(a_up, 32)-test5(a_down, 32))/0.02

1.7053025658242404e-10

In [250]:
t_func = @benchmark test5(a, 32)

BenchmarkTools.Trial: 7663 samples with 1 evaluation.
 Range (min … max):  538.800 μs …   2.645 ms  ┊ GC (min … max): 0.00% … 61.26%
 Time  (median):     606.900 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   645.689 μs ± 137.200 μs  ┊ GC (mean ± σ):  0.99% ±  4.11%

      ▄▆█▆▃▁                            ▁▁▁▂                     
  ▁▂▄▇██████▇▅▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▄▅▆█████▇▆▅▄▃▂▂▂▂▁▂▁▁▁▁▁▁▁ ▃
  539 μs           Histogram: frequency by time          800 μs <

 Memory estimate: 192.47 KiB, allocs estimate: 11.

In [251]:
t_funcgrad = @benchmark withgradient(x->test5(x, 32), a)

BenchmarkTools.Trial: 3465 samples with 1 evaluation.
 Range (min … max):  1.104 ms …   8.467 ms  ┊ GC (min … max): 0.00% … 75.63%
 Time  (median):     1.427 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.435 ms ± 243.262 μs  ┊ GC (mean ± σ):  1.16% ±  4.87%

                 ▄█▃                                           
  ▃▄▄▃▃▂▃▃▂▃▃▃▄▆████▆▅▅▅▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▂▁▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▂ ▃
  1.1 ms          Histogram: frequency by time        2.27 ms <

 Memory estimate: 435.05 KiB, allocs estimate: 44.

In [252]:
median(t_funcgrad).time/median(t_func).time

2.3512934585598946

## 1.7 - map2alm(map::HealpixMap{T,O,AA})

In [18]:
function test6(map::HealpixMap{T,O,AA}) where{T, O <:Order, AA<:AbstractArray{T,1}}
    a = map2alm(map)
    s = real(sum(a))
    return s
end

function test6_truegrad(map::HealpixMap{T,O,AA}) where{T, O <:Order, AA<:AbstractArray{T,1}}
    a = map2alm(map)
    a₁ = Alm(a.lmax, a.mmax, ones(numberOfAlms(a.lmax)))
    return conj(adjoint_map2alm(a₁, map.resolution.nside))
end

test6_truegrad (generic function with 1 method)

In [19]:
f, df = withgradient(x->test6(x), m)

(val = -1.4319394493171096, grad = ([-0.02692023030367345, 0.258392210542296, 0.25839221054229605, -0.02692023030367345, -0.03179509732433193, -0.05716981788125061, -0.16460786430164276, 0.2241705199020596, 0.22417051990205966, -0.1646078643016428  …  -0.0006017396164057321, -0.00046751495885258873, -0.0004675149588525886, -0.0006017396164057322, 0.0015007481043165407, 0.001820785408758422, -0.00040828713773921393, 0.0012085365526582409, 0.0012085365526582409, -0.0004082871377392134],))

In [20]:
df_true = test6_truegrad(m);

In [21]:
df[1][10]

-0.1646078643016428

In [22]:
df_true[10]

-0.1646078643016428

In [277]:
(test6(m_up)-test6(m_down))/0.02

-0.13627852625077708

In [265]:
eps = zeros(length(m))
eps[25]+=0.01
m_up = HealpixMap{Float64, RingOrder}(m + eps)
m_down = HealpixMap{Float64, RingOrder}(m - eps);

In [125]:
(test6(m_up)-test6(m_down))/0.02

-0.00020413183667855606

In [ ]:
median(t_funcgrad).time/median(t_func).time

## 1.8 - from_alm_to_healpix_alm(alm, l_max, nbin)

In [7]:
alms = from_healpix_alm_to_alm([a,a1], 2047, 1);

In [8]:
function test7(alms, lmax, nbin)
    hpix_alms = from_alm_to_healpix_alm(alms, lmax, nbin)
    s=0.
    for i in 1:2*nbin
        s += real(sum(hpix_alms[i]))
    end
    return s
end

test7 (generic function with 1 method)

In [9]:
f, df = withgradient(x->test7(x, 2047, 1), alms)

(val = 7.431646906538221, grad = ([[1.0; 1.0;;], [1.0 1.0 0.0; 1.0 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0]  …  [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0], [1.0 1.0 … 1.0 0.0; 1.0 1.0 … 1.0 0.0]],))

In [10]:
hpix_df = from_alm_to_healpix_alm(df[1],2047,1);

In [11]:
hpix_df[1].alm[11]

1.0 + 0.0im

In [12]:
alms1_up = from_healpix_alm_to_alm([a_up,a1], 2047, 1)
alms1_down = from_healpix_alm_to_alm([a_down,a1], 2047, 1);

In [13]:
(test7(alms1_up,2047,1)-test7(alms1_down,2047,1))/0.02

0.9999999999999787

In [76]:
t_func = @benchmark test7(alms, 2047, 1)

BenchmarkTools.Trial: 5 samples with 1 evaluation.
 Range (min … max):  1.065 s …   1.185 s  ┊ GC (min … max):  0.00% … 11.79%
 Time  (median):     1.175 s              ┊ GC (median):    10.89%
 Time  (mean ± σ):   1.155 s ± 50.035 ms  ┊ GC (mean ± σ):   9.20% ±  5.06%

  █                                                 █ ██  █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁██▁▁█ ▁
  1.07 s         Histogram: frequency by time        1.18 s <

 Memory estimate: 609.98 MiB, allocs estimate: 27394576.

In [77]:
t_funcgrad = @benchmark withgradient(x->test7(x, 2047, 1), alms)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.273 s …    1.557 s  ┊ GC (min … max):  8.75% … 8.86%
 Time  (median):     1.433 s               ┊ GC (median):     8.70%
 Time  (mean ± σ):   1.424 s ± 150.113 ms  ┊ GC (mean ± σ):  10.90% ± 4.12%

  █        █                                              ██  
  █▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██ ▁
  1.27 s         Histogram: frequency by time         1.56 s <

 Memory estimate: 876.33 MiB, allocs estimate: 27418241.

In [78]:
median(t_funcgrad).time/median(t_func).time

1.2198515989409893

## 1.9 - x_vec2vecmat(vec_x::Vector{Float64}, lmax::Int64, nbin::Int64)

In [14]:
vec_alms = x_vecmat2vec(alms, 2047, 1);

In [15]:
function test8(vec_x::Vector{Float64}, lmax::Int64, nbin::Int64)
    x = x_vec2vecmat(vec_x, lmax, nbin)
    return test7(x, lmax, nbin)
end

test8 (generic function with 1 method)

In [16]:
f, df = withgradient(x->test8(x, 2047, 1), vec_alms)

(val = 7.431646906538221, grad = ([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0  …  0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0],))

In [17]:
hpix_df = from_alm_to_healpix_alm(x_vec2vecmat(df[1], 2047, 1), 2047, 1)

2-element Vector{Any}:
 Alm{ComplexF64, Vector{ComplexF64}}(ComplexF64[1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im  …  1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im], 2047, 2047, 4095)
 Alm{ComplexF64, Vector{ComplexF64}}(ComplexF64[1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im  …  1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im, 1.0 + 0.0im], 2047, 2047, 4095)

In [18]:
hpix_df[1].alm[11]

1.0 + 0.0im

In [19]:
alms1_up = x_vecmat2vec(from_healpix_alm_to_alm([a_up,a1], 2047, 1), 2047, 1)
alms1_down = x_vecmat2vec(from_healpix_alm_to_alm([a_down,a1], 2047, 1), 2047, 1);

In [20]:
(test8(alms1_up,2047,1)-test8(alms1_down,2047,1))/0.02

0.9999999999999787

In [183]:
t_func = @benchmark test8(vec_alms, 2047, 1)

BenchmarkTools.Trial: 46 samples with 1 evaluation.
 Range (min … max):   66.675 ms … 317.861 ms  ┊ GC (min … max):  0.00% … 68.64%
 Time  (median):      88.751 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   109.099 ms ±  61.292 ms  ┊ GC (mean ± σ):  21.92% ± 22.30%

  ▁█▄▁ █  ▁                                                      
  ████▁█▇▁█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▄▄▄▁▁▁▁▁▁▁▄▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  66.7 ms          Histogram: frequency by time          318 ms <

 Memory estimate: 128.24 MiB, allocs estimate: 3601.

In [184]:
t_funcgrad = @benchmark withgradient(x->test8(x, 2047, 1), vec_alms)

BenchmarkTools.Trial: 14 samples with 1 evaluation.
 Range (min … max):  193.109 ms … 577.337 ms  ┊ GC (min … max):  0.00% … 54.22%
 Time  (median):     376.712 ms               ┊ GC (median):    36.53%
 Time  (mean ± σ):   359.261 ms ± 119.538 ms  ┊ GC (mean ± σ):  33.00% ± 22.48%

  ▁ ▁  ▁ ▁      ▁        ▁ ▁      ▁█▁      ▁            ▁     ▁  
  █▁█▁▁█▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█▁█▁▁▁▁▁▁███▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁█ ▁
  193 ms           Histogram: frequency by time          577 ms <

 Memory estimate: 458.49 MiB, allocs estimate: 25199.

In [185]:
median(t_funcgrad).time/median(t_func).time

4.244593575952469

## 1.10 - L_vec2vecmat(vec_L::Vector{Float64}, lmax::Int64, nbin::Int64)

In [35]:
vec_L = L_vecmat2vec([Matrix(gen_L[i].L) for i in 1:lmax+1], lmax, nbin);

LoadError: UndefVarError: `lmax` not defined

In [36]:
function test9(vec_L::Vector{Float64}, lmax::Int64, nbin::Int64)
    L = L_vec2vecmat(vec_L, lmax, nbin)
    return sum(sum(L))
end

test9 (generic function with 1 method)

In [312]:
f, df = withgradient(l->test9(l, lmax, nbin), vec_L)

(val = 57.079849470479104, grad = ([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],))

In [316]:
t_func = @benchmark test9(vec_L, lmax, nbin)

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.400 μs … 489.829 μs  ┊ GC (min … max):  0.00% … 98.22%
 Time  (median):     4.543 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.409 μs ±  17.198 μs  ┊ GC (mean ± σ):  11.77% ±  3.67%

  ▄██▆▄▂▁                                       ▁▁  ▁▁▁ ▁▁    ▂
  ████████▆▇▅▅▅▅▁▄▅▄▅▄▄▃▃▁▆██▇▄▅▄▃▁▃▃▄▁▅▄▄▁▄▃▁▃▇███████████▇▇ █
  4.4 μs       Histogram: log(frequency) by time      7.79 μs <

 Memory estimate: 9.20 KiB, allocs estimate: 128.

In [317]:
t_funcgrad = @benchmark withgradient(l->test9(l, lmax, nbin), vec_L)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  12.900 μs …  3.534 ms  ┊ GC (min … max):  0.00% … 98.63%
 Time  (median):     13.900 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   17.647 μs ± 88.231 μs  ┊ GC (mean ± σ):  13.08% ±  2.61%

  ▃▇██▅▃▁                        ▁▁                           ▂
  ███████▇▆▆▆▅▄▆▆▆▆▆▆▇▇▇▇▇██████▇███▇▇█▇▇████▆▇██▇▆▇▆▆▆▇▆▆▅▅▄ █
  12.9 μs      Histogram: log(frequency) by time      32.4 μs <

 Memory estimate: 33.89 KiB, allocs estimate: 278.

In [318]:
median(t_funcgrad).time/median(t_func).time

3.0597484276729556

## 1.11 - vectorK_vec2vecmat

In [21]:
n_free_params = 9
nbin = 2
lmax = 2047
vec_K = randn(n_free_params*(lmax+1));

In [22]:
function test10(all_k, nbin, lmax, freeparam_n)
    K = vectorK_vec2vecmat(all_k, nbin, lmax, freeparam_n)
    return sum(sum(K))
end

test10 (generic function with 1 method)

In [23]:
f, df = withgradient(x->test10(x, nbin, lmax, n_free_params), vec_K)

(val = 143.5718169815428, grad = ([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],))

In [24]:
df[1][5]

1.0

In [25]:
ϵ = zeros(length(vec_K))
ϵ[5] += 0.01
k_plus = deepcopy(vec_K) + ϵ
k_min = deepcopy(vec_K) -ϵ;

(test10(k_plus, nbin, lmax, n_free_params)-test10(k_min, nbin, lmax, n_free_params))/0.02

1.0000000000005116

In [51]:
t_func = @benchmark test10(vec_K, nbin, lmax, n_free_params)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  422.900 μs …   1.335 ms  ┊ GC (min … max): 0.00% … 52.47%
 Time  (median):     430.200 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   450.224 μs ± 100.774 μs  ┊ GC (mean ± σ):  3.03% ±  8.38%

  █▅▂▂      ▁                                                   ▁
  █████▇▇▆▆██▆▅▅▄▃▁▄▁▃▁▁▁▃▄▁▃▄▁▁▁▃▃▁▁▁▁▃▁▁▁▁▁▁▃▆▆▄▄▃▃▄▃▄▄▆▆▆▇▇█ █
  423 μs        Histogram: log(frequency) by time       1.06 ms <

 Memory estimate: 1.02 MiB, allocs estimate: 6145.

In [52]:
t_funcgrad = @benchmark withgradient(x->test10(x, nbin, lmax, n_free_params), vec_K)

BenchmarkTools.Trial: 1881 samples with 1 evaluation.
 Range (min … max):  2.239 ms … 61.874 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     2.303 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.653 ms ±  1.564 ms  ┊ GC (mean ± σ):  5.59% ± 11.73%

  █▇▃▃▃▁               ▁▁                                     
  █████████▇▆▇▇▅▆▅▆▅▆▇▆███▇▇▄▇▆▅▆▇▆▄▅▃▅▇█▇▆▇▅▆▆▆▇▅▆▅▃▁▄▃▃▄▁▃ █
  2.24 ms      Histogram: log(frequency) by time     5.42 ms <

 Memory estimate: 2.88 MiB, allocs estimate: 79916.

In [53]:
median(t_funcgrad).time / median(t_func).time

5.353324035332403

## 1.12 - vector_from_k_to_L

In [26]:
function test11(all_k, nbin, lmax, freeparam_n)
    L = vector_from_k_to_L(all_k, nbin, lmax, freeparam_n)
    return sum(sum(L))
end

test11 (generic function with 1 method)

In [27]:
f, df = withgradient(x->test11(x, nbin, lmax, n_free_params), vec_K)

(val = 12429.04930899319, grad = ([0.8795170614959338, 0.6666399989334546, 3.8282519263415624, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.4851751932018507  …  1.0, 0.8854953857095382, 0.6276909989373958, 1.4199991294151255, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],))

In [28]:
ϵ = zeros(length(vec_K))
ϵ[2] += 0.01
k_plus = deepcopy(vec_K) + ϵ
k_min = deepcopy(vec_K) -ϵ;

(test11(k_plus, nbin, lmax, n_free_params)-test11(k_min, nbin, lmax, n_free_params))/0.02

0.6666511097137118

In [112]:
t_func = @benchmark test11(vec_K, nbin, lmax, n_free_params)

BenchmarkTools.Trial: 9966 samples with 1 evaluation.
 Range (min … max):  465.400 μs …   1.750 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     474.200 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   498.644 μs ± 117.807 μs  ┊ GC (mean ± σ):  3.07% ± 8.49%

  █▅▂▂         ▁                                                ▁
  █████▇▇▆▆▅▅▅▇█▆▅▄▃▅▁▃▃▄▁▃▁▄▃▃▃▃▄▃▁▁▄▄▁▁▄▇▆▃▅▄▁▃▃▁▅▁▁▃▃▁▄▇▇▆▇█ █
  465 μs        Histogram: log(frequency) by time       1.18 ms <

 Memory estimate: 1.02 MiB, allocs estimate: 6145.

In [113]:
t_funcgrad = @benchmark withgradient(x->test11(x, nbin, lmax, n_free_params), vec_K)

BenchmarkTools.Trial: 1857 samples with 1 evaluation.
 Range (min … max):  2.475 ms …   5.469 ms  ┊ GC (min … max): 0.00% … 33.43%
 Time  (median):     2.513 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.688 ms ± 530.989 μs  ┊ GC (mean ± σ):  4.79% ± 10.71%

  █▇▃▁                                            ▁        ▁   
  ████▆▆▆▅▅▄▇▆▅▃▅▄▁▅▃▁▁▃▁▆▅▄▃▄▁▅▁▃▃▃█▃▃▁▁▁▃▁▃▁▁▁▃██▆▅▁▄▆▆▄▄█▇ █
  2.48 ms      Histogram: log(frequency) by time      4.73 ms <

 Memory estimate: 2.88 MiB, allocs estimate: 79916.

In [114]:
median(t_funcgrad).time / median(t_func).time

5.298397300716997

# 2 - INFERENCE FUNCTIONS TESTS

## 2.1 - Generate fictional data

In [2]:
nbin = 1
lmax = 127
q = 0#-0.5*(2*nbin+1)
nside = 64
npix = nside2npix(nside)
ε = 0.27/sqrt(2*3)  #(0.3/sqrt(36))
N_noise = ε * I
inv_N_noise = inv(N_noise);

In [3]:
@model function data_generator(l_max, n_tomo)
    L = Vector{Matrix{Float64}}(undef, l_max+1)# a vector of l_max+1 matrices, each of them will 2n_tomo x 2n_tomo elements
    x = Vector{Matrix{Float64}}(undef, l_max+1)# a vector of l_max+1 matrices, the i-th matrix has (2ntomo x i) elements
    
    for i in 1:l_max+1
        L[i] ~ LKJCholesky(2*n_tomo, 1)
        x[i] ~ arraydist([MvNormal(zeros(2*n_tomo), Matrix{Float64}(I,2*n_tomo,2*n_tomo)) for p in -(i-1):(i-1)])
    end
end

data_generator (generic function with 2 methods)

### 2.1.1 - Fictional measured data

In [4]:
# x matrices
gen_data = rand(data_generator(lmax, nbin))
gen_L = collect(gen_data[i] for i in 1:2:2*(lmax+1));
vec_x = x_vecmat2vec(collect(gen_data[i] for i in 2:2:2*(lmax+1)), lmax, nbin)

# K and L matrices
free_param_n = (2*nbin - 1)*(nbin + 1)
vec_K = rand(Uniform(-1,1),(lmax+1)*free_param_n);
#L = vector_from_k_to_L(vec_K, nbin, lmax, free_param_n);

In [5]:
#gen_C, gen_a = global_from_Cholesky(Cholesky.(LowerTriangular.(L)), collect(gen_data[i] for i in 2:2:2*(lmax+1)))
gen_C, gen_a = global_from_Cholesky(gen_L, collect(gen_data[i] for i in 2:2:2*(lmax+1)))
hpix_gen_alms = from_alm_to_healpix_alm(gen_a, lmax, nbin)
gen_maps = Array{HealpixMap}(undef, 2*nbin)
for i in 1:2*nbin
    gen_maps[i] = alm2map(hpix_gen_alms[i], nside)
end

### 2.1.2 - Initial condition

In [6]:
θₜ = vcat(vec_x, vec_K);

In [7]:
vec_x = 0
gen_C = 0
gen_a = 0
hpix_gen_alms = 0

0

In [8]:
#θ₀ = rand(MvNormal(θₜ, Matrix(I, length(θₜ), length(θₜ))));
θ₀=θₜ;

In [9]:
K₀ = vectorK_vec2vecmat(θ₀[2*nbin*(2*numberOfAlms(lmax)-(lmax+1))+1:end], nbin, lmax, free_param_n)
x₀ = x_vec2vecmat(θ₀[1:2*nbin*(2*numberOfAlms(lmax)-(lmax+1))], lmax, nbin)
L₀ = vector_from_k_to_L(θ₀[2*nbin*(2*numberOfAlms(lmax)-(lmax+1))+1:end], nbin, lmax, free_param_n);

## 2.2 - Negloglikelihood

$$-\log{(\mathcal{L}(\mathrm{L},\mathbf{x}))} = \sum_{i=1}^{2n_{bins}} \left[ \frac{1}{2}(\mathbf{d}-\mathrm{Y}\mathrm{L} \mathbf{x})^{\mathrm{T}}\mathrm{N}^{-1}(\mathbf{d}-\mathrm{Y}\mathrm{L} \mathbf{x}) \right]$$

In [10]:
function negloglikelihood(L::Vector{Matrix{Float64}}, x::Vector{Matrix{Float64}})
    loglike = 0.
    a = Chol_Lx2a(x, L)
    alms = from_alm_to_healpix_alm(a, lmax, nbin)
    for i in 1:2*nbin
        d̃ = alm2map(alms[i], nside)
        loglike += 0.5*transpose(gen_maps[i]-d̃)*inv_N_noise*(gen_maps[i]-d̃)
    end
    return loglike
end

negloglikelihood (generic function with 1 method)

### 2.2.1 -  $L$ variables test

In [21]:
ℒ, dℒ_dL = withgradient(ℓ->negloglikelihood(ℓ, x₀), L₀);

In [22]:
dℒ_dL[1][1]

2×2 Matrix{Float64}:
    -2.13288      0.100295
 61622.9      -2897.73

In [23]:
L₊ = deepcopy(L₀)
L₋ = deepcopy(L₀)
L₊[1][1,2] += 0.01
L₋[1][1,2] -= 0.01
(negloglikelihood(L₊, x₀) - negloglikelihood(L₋, x₀))/0.02

0.10029971599578857

In [24]:
dℒ_dL[1][10]

2×2 Matrix{Float64}:
 -1.96355e5  -12408.9
 -2.38297e5      -3.60509e5

In [25]:
L₊ = deepcopy(L₀)
L₋ = deepcopy(L₀)
L₊[10][2,2] += 0.01
L₋[10][2,2] -= 0.01
(negloglikelihood(L₊, x₀) - negloglikelihood(L₋, x₀))/0.02

-360508.7184369564

#### Benchmarks

In [27]:
t_func = @benchmark negloglikelihood(L₀, x₀)

BenchmarkTools.Trial: 1356 samples with 1 evaluation.
 Range (min … max):  2.846 ms …  10.479 ms  ┊ GC (min … max): 0.00% … 42.61%
 Time  (median):     3.372 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.672 ms ± 780.283 μs  ┊ GC (mean ± σ):  3.86% ±  7.82%

     ▃▇█▄▄▃                                                    
  ▃▆▆████████▇▆▅▆▅▅▄▅▅▄▄▄▅▄▄▃▄▄▃▄▄▃▃▃▃▃▃▃▃▂▃▃▃▂▂▂▂▂▁▂▃▂▂▂▂▂▂▂ ▃
  2.85 ms         Histogram: frequency by time        6.17 ms <

 Memory estimate: 5.55 MiB, allocs estimate: 444.

In [28]:
t_funcgrad = @benchmark withgradient(ℓ->negloglikelihood(ℓ, x₀), L₀)

BenchmarkTools.Trial: 516 samples with 1 evaluation.
 Range (min … max):  7.020 ms … 33.668 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     9.492 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.683 ms ±  1.877 ms  ┊ GC (mean ± σ):  3.86% ± 5.47%

                ▄█▁                                           
  ▃▄▃▄▃▃▄▆▅▇█▆▅▆███▅▄▄▅▆▅▅▄▄▃▂▃▂▃▂▁▁▃▂▂▁▂▂▂▁▂▂▂▁▁▁▂▁▁▁▂▁▂▁▁▂ ▃
  7.02 ms        Histogram: frequency by time        16.5 ms <

 Memory estimate: 14.50 MiB, allocs estimate: 3130.

In [29]:
median(t_funcgrad).time/median(t_func).time

2.814759348773761

### 2.2.2 -  $x$ variables test

In [84]:
ℒ, dℒ_dx = withgradient(x->negloglikelihood(L₀, x), x₀);

In [85]:
dℒ_dx[1][3]

2×5 Matrix{Float64}:
 -314.636  -13573.9   -41116.1   40789.2  -41972.6
  387.26     3835.83   14904.9  -24721.0   18052.4

In [86]:
x₊ = deepcopy(x₀)
x₋ = deepcopy(x₀)
x₊[3][2,1] += 0.01
x₋[3][2,1] -= 0.01
(negloglikelihood(L₀, x₊) - negloglikelihood(L₀, x₋))/0.02

387.2601270675659

In [87]:
dℒ_dx[1][5]

2×9 Matrix{Float64}:
 -531.421  -14487.9    -13069.5   -53905.9  …  -4043.03  19840.0   12476.2
 -606.7      -743.016   -9025.31  -20528.6     -5409.59   5992.68   3473.65

In [88]:
x₊ = deepcopy(x₀)
x₋ = deepcopy(x₀)
x₊[5][2,7] += 0.01
x₋[5][2,7] -= 0.01
(negloglikelihood(L₀, x₊) - negloglikelihood(L₀, x₋))/0.02

-5409.5940083265305

In [89]:
x₊ = deepcopy(x₀)
x₋ = deepcopy(x₀)
x₊[5][2,8] += 0.01
x₋[5][2,8] -= 0.01
(negloglikelihood(L₀, x₊) - negloglikelihood(L₀, x₋))/0.02

5992.684578895569

#### Benchmarks

In [90]:
t_funcgrad = @benchmark withgradient(x->negloglikelihood(L₀, x), x₀)

BenchmarkTools.Trial: 515 samples with 1 evaluation.
 Range (min … max):  8.135 ms … 17.712 ms  ┊ GC (min … max): 0.00% … 23.35%
 Time  (median):     9.229 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.708 ms ±  1.366 ms  ┊ GC (mean ± σ):  4.28% ±  5.80%

      ▄█▆       ▂▃                                            
  ▄▅▃▄████▅▂▃▃▆███▆▅▃▃▃▃▃▄▃▄▂▂▂▁▁▂▁▁▂▂▁▂▂▂▃▂▃▂▁▁▁▁▁▃▁▁▂▁▁▂▂▂ ▃
  8.14 ms        Histogram: frequency by time        15.4 ms <

 Memory estimate: 14.50 MiB, allocs estimate: 3105.

In [91]:
median(t_funcgrad).time/median(t_func).time

2.7827594524513057

## 2.3 - Neglogprior1

$$-\log{(\mathcal{P}_1(\mathbf{x}))} = \sum_{i=1}^{2n_{bins}} \left[ \frac{1}{2}\sum_{\ell}\sum_{m}\mathbf{x}_{\ell m}^{\mathrm{T}}\mathbf{x}_{\ell m} \right] $$

function neglogprior1(x::Vector{Matrix{Float64}})
    logprior = 0.
    hlpix_x = from_alm_to_healpix_alm(x, lmax, nbin)
    for i in 1:2*nbin
        logprior += 0.5*sum(alm_prod(hlpix_x[i],hlpix_x[i]))
    end
    return real(logprior)
end

In [14]:
function neglogprior1(x::Vector{Float64})
    logprior =  0.5*sum(x .* x)
    return logprior
end

@adjoint function neglogprior1(x::Vector{Float64})
    y = neglogprior1(x)
    function neglogprior1_pullback(ȳ)
        x̄ = deepcopy(x) .* ȳ
        return (x̄,)
    end
    return y, neglogprior1_pullback
end

In [15]:
neglogprior1(vec_x)

4.1948779658138e6

In [16]:
𝒫₁, d𝒫₁ = withgradient(x->neglogprior1(x), vec_x)

(val = 4.1948779658138e6, grad = ([-1.2754964526766959, -1.1556866267577786, -1.0802775118673145, -0.9293493753605436, 0.6163653072876508, -0.3231044218214436, -1.1587214028308848, 0.567926042054044, -0.41019532806641984, 0.2616371697452952  …  -0.9939237672753833, -0.12913656753819422, -0.7899582912390452, -0.028665993274613073, -0.12277288516835984, -0.15797805785254357, -0.25911525730528434, -0.5850703669815005, 0.4093646904478641, -1.617692738395248],))

In [19]:
d𝒫₁[1][33]

-1.5159221480935425

In [20]:
x₊ = deepcopy(vec_x)
x₋ = deepcopy(vec_x)
x₊[33] += 0.01
x₋[33] -= 0.01
(neglogprior1(x₊) - neglogprior1(x₋))/0.02

-1.5159221831709146

In [21]:
d𝒫₁[1][6]

-0.3231044218214436

In [23]:
x₊ = deepcopy(vec_x)
x₋ = deepcopy(vec_x)
x₊[6] += 0.01
x₋[6] -= 0.01
(neglogprior1(x₊) - neglogprior1(x₋))/0.02

-0.3231044393032789

#### Benchmarks

In [301]:
t_func = @benchmark neglogprior1(x₀)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  28.900 μs …  3.198 ms  ┊ GC (min … max): 0.00% … 97.92%
 Time  (median):     29.800 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   32.918 μs ± 61.521 μs  ┊ GC (mean ± σ):  3.68% ±  1.96%

  ▄▇█▇▄▁▁▁   ▁▁▁▁▁▁  ▁▁ ▁▂▁ ▁▂▁  ▁▁▁     ▁                    ▂
  ██████████▇█████████████████████████████▇▆▇█▇▅▆▇▇▆▇▇▅▆▅▆▃▅▄ █
  28.9 μs      Histogram: log(frequency) by time      48.1 μs <

 Memory estimate: 23.12 KiB, allocs estimate: 225.

In [302]:
t_funcgrad = @benchmark withgradient(x->neglogprior1(x), x₀)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  43.300 μs …  2.470 ms  ┊ GC (min … max): 0.00% … 95.65%
 Time  (median):     44.950 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   50.924 μs ± 82.507 μs  ┊ GC (mean ± σ):  5.65% ±  3.45%

  ▇█▄▂▂        ▁▁ ▁ ▁▁ ▁▁                                     ▂
  ██████▇▆▅▄▅▅▆████████████▇▆▆▆▅▅▅▄▄▃▆▄▄▅▅▃▃▄▃▃▅▁▁▁▄▃▃▁▃▁▃▁▃▄ █
  43.3 μs      Histogram: log(frequency) by time       126 μs <

 Memory estimate: 64.25 KiB, allocs estimate: 320.

In [303]:
median(t_funcgrad).time/median(t_func).time

1.5083892617449663

## 2.4 - Neglogprior2

$$-\log{(\mathcal{P}_2(K))} = \sum_{j=1}^{2n_{bins}} \left[ \sum_{\ell}\sum_{i=1}^{n}(i-2-n-2q){\mathrm{K}_{\ell , ii}} \right] $$

In [14]:
function neglogprior2(K::Vector{Matrix{Float64}})
    c = 2 + (2*nbin) + (2*q)
    logprior = 0.
    for i in 2:2*nbin
        cons = i-c
        for l in 1:lmax+1
            logprior += cons*K[l][i,i]
        end
    end
    return logprior
end

@adjoint function neglogprior2(K::Vector{Matrix{Float64}})
    y = neglogprior2(K)
    function neglogprior2_pullback(ȳ)
        c = 2 + (2*nbin) + (2*q)
        single_K̄ = diagm([i-c for i in 1:2*nbin])
        single_K̄[1,1] = 0.0
        K̄ = [single_K̄ for i in 1:lmax+1]
        return (K̄,)
    end
    return y, neglogprior2_pullback
end

In [237]:
𝒫₂, d𝒫₂ = withgradient(K->neglogprior2(K), K₀)

(val = -6.303722096016132, grad = ([[0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0]  …  [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0], [0.0 0.0; 0.0 -2.0]],))

In [238]:
d𝒫₂[1][6]

2×2 Matrix{Float64}:
 0.0   0.0
 0.0  -2.0

In [239]:
L₊ = deepcopy(L₀)
L₋ = deepcopy(L₀)
L₊[6][1,2] += 0.01
L₋[6][1,2] -= 0.01
(neglogprior2(L₊) - neglogprior2(L₋))/0.02

0.0

In [240]:
d𝒫₂[1][4]

2×2 Matrix{Float64}:
 0.0   0.0
 0.0  -2.0

In [241]:
L₊ = deepcopy(L₀)
L₋ = deepcopy(L₀)
L₊[4][2,2] += 0.01
L₋[4][2,2] -= 0.01
(neglogprior2(L₊) - neglogprior2(L₋))/0.02

-2.000000000001023

#### Benchmark

In [15]:
t_func = @benchmark neglogprior2(K₀)

BenchmarkTools.Trial: 3434 samples with 1 evaluation.
 Range (min … max):  316.800 μs … 247.055 ms  ┊ GC (min … max):  0.00% … 99.72%
 Time  (median):     393.900 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.475 ms ±  15.463 ms  ┊ GC (mean ± σ):  71.67% ±  6.80%

  ▃     █▅▆▅▃▂▁                                                 ▁
  ██▅▅▁▅████████▆▇▄▅▃▅▁▄▄▄▃▅▄▄▄▃▄▁▃▄▄▅▆▅▇██▇▅▁▃▃▃▅▄▁▁▃▄▁▁▃▁▁▄▄▃ █
  317 μs        Histogram: log(frequency) by time          1 ms <

 Memory estimate: 208.14 KiB, allocs estimate: 11270.

In [16]:
t_funcgrad = @benchmark withgradient(K->neglogprior2(K), K₀)

BenchmarkTools.Trial: 1635 samples with 1 evaluation.
 Range (min … max):  420.800 μs … 286.276 ms  ┊ GC (min … max):  0.00% … 99.72%
 Time  (median):     601.500 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     3.178 ms ±  24.686 ms  ┊ GC (mean ± σ):  79.19% ± 10.12%

  █                                                              
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▂
  421 μs           Histogram: frequency by time          215 ms <

 Memory estimate: 465.17 KiB, allocs estimate: 13350.

In [17]:
median(t_funcgrad).time/median(t_func).time

1.527037319116527